# Extract Evaluation Q&A Pairs

This notebook extract Q&A from evaluation raw data generated by the LLM

## Import Packages

In [5]:
import pandas as pd
import re
import pickle

## Define Functions

In [6]:
def extract_eval_qa(text):
    q_pattern = re.compile(r'^\s*Q[0-9]:(.*?)(?=^\s*A\d*:|\Z)', re.MULTILINE | re.DOTALL)
    a_pattern = re.compile(r'^\s*A\d*:\s*(.+?)(?=^\s*Q\d+:|\Z)', re.MULTILINE | re.DOTALL)
    questions = re.findall(q_pattern, text)
    answers = re.findall(a_pattern,text)
    if len(questions)!=len(answers):
        raise Exception(f"LLM response does not follow expected format:\n{text}")
    else:
        qa_list = [{'question':q.strip(),'answer':a.strip()} for q,a in zip(questions,answers)]
    return qa_list

## Load data

In [7]:
with open('../../data/intermediate/eval_qa.pkl','rb') as f:
    data = pickle.load(f)

## Extract Q&A

In [8]:
# Extract qa
data['qa'] = data['evaluation_qa'].apply(lambda x: extract_eval_qa(x))

# Explode Q&A pairs
exploded_df = data.explode(column=['qa'])

# Remove some noise data that didn't produce qa pairs
exploded_df = exploded_df[~exploded_df['qa'].isna()]

# Generate question and answer columns
exploded_df['question'] = exploded_df['qa'].apply(lambda x: x['question'])
exploded_df['answer'] = exploded_df['qa'].apply(lambda x: x['answer'])

## Create eval dataset

In [ ]:
# Create eval data
eval_data = exploded_df[['question','answer']].reset_index(drop=True)

## Save data

In [ ]:
eval_data.to_pickle('../../data/eval_data.pkl')
eval_data.to_csv('../../data/eval_data.csv', index=False)